### Investigate the convertion from CARMA params to Celerite terms

<br>**Author(s):** Weixiang Yu
<br>**Last run:** 06-17-20
<br>**Short description:** Two tasks: 1. Examine the built in CARMA solver of Celertie; 2. If not working, develope a new convertion fuction.

In [2]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# see if local stores mpl style, else use from src
try:
    plt.style.use('yu_basic')
except:
    mpl.rc_file('../../src/vis/mpl/yu_basic.rc')

pd.set_option('display.max_columns', 999)
%matplotlib inline

In [3]:
import celerite
from celerite import GP, terms
from celerite.solver import get_kernel_value, CARMASolver

### 1. Test Celerite CARMA Solver
The first step is to see whether the built in CARMASolver give the correct convertion by examing the the ACF of a input CARMA model. Given an overdamped 2nd order CARMA model (two real roots), we know the ACF pairs must have opposite signs. And thus the corresponding Celerite real terms must also have opposite signs. 
<br> Here we are using a CARMA(2,0) model, with AR parameters: [2, 0.8] and $\sigma$ = 1

In [4]:
# compute the ACF of the input model
arroots = np.roots([1, 2, 0.8])
sigma = 1
print (f'AR roots: {arroots}')

AR roots: [-1.4472136 -0.5527864]


In [10]:
r1, r2 = arroots
acf_1 = sigma**2/(-2*r1*(r2-r1)*(r2+r1))
acf_2 = sigma**2/(-2*r2*(r1-r2)*(r1+r2))
print(f'ACF for root 1: {acf_1}')
print(f'ACF for root 2: {acf_2}')

ACF for root 1: -0.19313562148434218
ACF for root 2: 0.5056356214843422


#### Now see what Celerite returns

In [16]:
carma_solver = CARMASolver(0, np.array([2, 0.8]), np.array([]))
carma_solver.get_celerite_coeffs()

(array([], dtype=float64),
 array([], dtype=float64),
 array([0.03040503]),
 array([-0.01364223]),
 array([1.11277046]),
 array([-2.48008024]))

We can see `Celerite` returns a complex kernel instead of two real kernals with opposite signs in $a_{i}$. 
<br> The conclusion is that we need to write our own mapping function in order to fit CARMA processes with Celerite

### 2. CARMA ACF

In [64]:
# construct/test using CARMA(2,1), define in Kasliwal's notation and add to Kelly's
arparam = np.array([2, 0.8])
maparam = np.array([1])

sigma = maparam[0]
n_maparam = np.array([x/sigma for x in maparam])

In [65]:
# get roots
arroots = np.roots(np.append([1], arparam))
maroots = np.roots(n_maparam[::-1])
print(f'AR roots: {arroots}')
print(f'MA roots: {maroots}')

AR roots: [-1.4472136 -0.5527864]
MA roots: []


In [66]:
# variable for order
p = len(arparam)
q = len(maparam) -1

#### 2.1 Vectorized CARMA ACF

In [67]:
# get acf in a vectorized way
num_0 = 0
num_1 = 0
denum = -2*arroots.real

for k in range(q+1):
    num_0 += maparam[k]*np.power(arroots, k)
    num_1 += maparam[k]*np.power(-arroots, k)

for j in range(1, p):
    root_idx = np.arange(p)
    root_k = arroots[np.roll(root_idx, j)]
    print(root_k)
    denum *= (root_k - arroots)*(np.conj(root_k) + arroots)

[-0.5527864 -1.4472136]


In [69]:
print(f'ACF for CARMA(2,0): {num_0*num_1/denum}')

ACF: [-0.19313562  0.50563562]


#### 2.2 Construct more test examples

### 3. From ACF to Celerite Terms